In [1]:
#
# SpiceyPy package:
#
import spiceypy.utils.support_types as stypes
import spiceypy
import os

def visibl():
    #
    # Local Parameters
    #
    METAKR = 'MEX_OPS.TM'
    SCLKID = -82
    TDBFMT = 'YYYY MON DD HR:MN:SC.### TDB ::TDB'
    MAXIVL = 1000
    MAXWIN = 2 * MAXIVL

    #
    # Load the meta-kernel.
    #
    os.chdir("/home/swalter/naif/MEX/kernels/mk")
    spiceypy.furnsh( METAKR )

    #
    # Assign the inputs for our search.
    #
    srfpt  = 'MARS'
    obsfrm = 'IAU_MARS'
    target = 'MEX'
    abcorr = 'NONE'
    start  = '2004 MAY 2 TDB'
    stop   = '2004 MAY 6 TDB'
    refval = 0.0
    relate = "=";
    crdsys = "RECTANGULAR";
    coord  = "Z";

    #
    # The adjustment value only applies to absolute extrema
    # searches; simply give it an initial value of zero
    # for this inequality search.
    #
    adjust = 0.0

    #
    # stepsz is the step size, measured in seconds, used to search
    # for times bracketing a state transition. Since we don't expect
    # any events of interest to be shorter than five minutes, and
    # since the separation between events is well over 5 minutes,
    # we'll use this value as our step size. Units are seconds.
    #
    stepsz = 300.0

    #
    # Display a banner for the output report:
    #
    print( '\n{:s}\n'.format('Inputs for target visibility search:')  )

    print( '   Target                       = {:s}'.format( target )  )
    print( '   Observation surface location = {:s}'.format( srfpt  )  )
    print( '   Observer\'s reference frame   = {:s}'.format( obsfrm )  )
    print( '   Latitude limit (degerees)    = {:f}'.format( refval )  )
    print( '   Aberration correction        = {:s}'.format( abcorr )  )
    print( '   Step size (seconds)          = {:f}'.format( stepsz )  )

    #
    # Convert the start and stop times to ET.
    #
    etbeg = spiceypy.str2et( start )
    etend = spiceypy.str2et( stop  )

    #
    # Display the search interval start and stop times
    # using the format shown below.
    #
    #    2004 MAY 06 20:15:00.000 (TDB)
    #
    btmstr = spiceypy.timout( etbeg, TDBFMT )
    print( '   Start time                   = '
           '{:s}'.format(btmstr) )

    etmstr = spiceypy.timout( etend, TDBFMT )
    print( '   Stop time                    = '
           '{:s}'.format(etmstr) )

    print( ' ' )

    #
    # Initialize the "confinement" window with the interval
    # over which we'll conduct the search.
    #
    cnfine = stypes.SPICEDOUBLE_CELL(2)
    spiceypy.wninsd( etbeg, etend, cnfine )

    #
    # In the call below, the maximum number of window
    # intervals gfposc can store internally is set to MAXIVL.
    # We set the cell size to MAXWIN to achieve this.
    #
    riswin = stypes.SPICEDOUBLE_CELL( MAXWIN )

    #
    # Now search for the time period, within our confinement
    # window, during which the apparent target has elevation
    # at least equal to the elevation limit.
    #
    spiceypy.gfposc( target, obsfrm, abcorr, srfpt,
                     crdsys, coord,  relate, refval,
                     adjust, stepsz, MAXIVL, cnfine, riswin )
    #
    # The function wncard returns the number of intervals
    # in a SPICE window.
    #
    winsiz = spiceypy.wncard( riswin )
    
    if winsiz == 0:
        print( 'No events were found.' )
    else:
        #
        # Display the visibility time periods.
        #
        print( '{0:s} {1:s} equator crossing times:\n'.format(target, srfpt ))
        for  i  in  range(winsiz):
            #
            # Fetch the start and stop times of
            # the ith interval from the search result
            # window riswin.
            #
            [intbeg, intend] = spiceypy.wnfetd( riswin, i )
            #
            # Convert the rise time to a TDB calendar string.
            #
            timstr = spiceypy.timout( intbeg, TDBFMT )
            #
            # Write the string to standard output.
            #
            if  i  ==  0:
                print( 'Equator crossing or start time:   {:s}'.format( timstr )          )
            else:
                print( 'Equator crossing time:            {:s}'.format( timstr )          )
            #
            # Convert the set time to a TDB calendar string.
            #
            timstr = spiceypy.timout( intend, TDBFMT )
            #
            # Write the string to standard output.
            #
            if  i  ==  (winsiz-1):
                print( 'Equator crossing or stop time:    {:s}'.format( timstr )          )
            #else:
            #    print( 'Visibility stop time:             {:s}'.format( timstr )          )
            #print( ' ' )
    spiceypy.unload( METAKR )

if __name__ == '__main__':
    visibl()


Inputs for target visibility search:

   Target                       = MEX
   Observation surface location = MARS
   Observer's reference frame   = IAU_MARS
   Latitude limit (degerees)    = 0.000000
   Aberration correction        = NONE
   Step size (seconds)          = 300.000000
   Start time                   = 2004 MAY 02 00:00:00.000 TDB
   Stop time                    = 2004 MAY 06 00:00:00.000 TDB
 
MEX MARS equator crossing times:

Equator crossing or start time:   2004 MAY 02 05:00:08.334 TDB
Equator crossing time:            2004 MAY 02 06:15:13.074 TDB
Equator crossing time:            2004 MAY 02 12:35:14.856 TDB
Equator crossing time:            2004 MAY 02 13:50:09.161 TDB
Equator crossing time:            2004 MAY 02 20:10:24.439 TDB
Equator crossing time:            2004 MAY 02 21:25:10.344 TDB
Equator crossing time:            2004 MAY 03 03:45:26.758 TDB
Equator crossing time:            2004 MAY 03 05:00:04.086 TDB
Equator crossing time:            2004 MAY 03 11